In [11]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_addons as tfa
import random
import nltk
import os
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report, roc_curve, roc_auc_score, confusion_matrix, accuracy_score, f1_score, cohen_kappa_score
from sklearn.model_selection import GroupKFold, train_test_split
from sklearn import tree, metrics

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, GlobalMaxPooling1D, Embedding, Dropout, Bidirectional, LSTM

from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

from imblearn.over_sampling import SMOTE


random.seed(20)

In [2]:
# import data
df = pd.read_csv('Data/Annotations_final.csv')

In [3]:
# split data for student level cross validation: one student is nested within one fold
group_dict = dict()
groups = np.array([])

for index, row in df.iterrows():
    s_id = row['created_by']
    if s_id not in group_dict:
        group_dict[s_id] = index
    groups = np.append(groups, group_dict[s_id])
    
# Set up the splitter with 5 splits
gkf = GroupKFold(n_splits = 5)

## Bag of Words (Unigram)

In [4]:
# bag of words + neural nets


# stemming
porter = PorterStemmer() # stemming recovers root words from plurals etc
stemmed_texts = []

X_text = df['annotation_text']

for answer in X_text:
    answer = ' '.join(porter.stem(word) for word in answer.split(' '))
    stemmed_texts.append(answer)

X, y = np.array(stemmed_texts), df['comment_process'] # CHANGE y HERE

vect = CountVectorizer(ngram_range=(1,1), max_features=1000, stop_words="english") #only unigram 313 features
#vect = CountVectorizer(ngram_range=(2,2), max_features=1000, stop_words="english") #only bigram 728 features

X = vect.fit_transform(X).toarray()


# set up storage arrays for each round of validation
roc_auc_scores = np.array([])
accuracy_scores = np.array([])

# split, train, test and store performance metrics
for train_index, test_index in gkf.split(X, y, groups=groups):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    
    model = Sequential()
    model.add(Dense(12, input_shape=(313,), activation='relu')) 
    model.add(Dense(8, activation='relu')) 
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', 
              optimizer='adam',
              metrics = ['acc']
             )
    
    num_epochs = 30
    batch_size = 10
    
    model.fit(
        X_train, 
        y_train, 
        epochs=num_epochs, 
        validation_split=0.1,
        shuffle=True, 
        batch_size=batch_size)
    
    predictions = model.predict(X_test)
    
    # compute some metrics and store them for averaging later on
    roc_auc_scores = np.append(roc_auc_scores, roc_auc_score(y_test, predictions))

# print mean scores for the 5-fold CV
print("average roc_auc score: ", np.round(roc_auc_scores.mean(), 3))
print("stdv roc_auc score: ", np.round(roc_auc_scores.std(), 3))
print("max roc_auc score: ", np.round(roc_auc_scores.max(), 3))


Epoch 1/30

17/17 [==============================] - 1s 9ms/step - loss: 0.6856 - acc: 0.6280 - val_loss: 0.6857 - val_acc: 0.6316
Epoch 2/30
17/17 [==============================] - 0s 2ms/step - loss: 0.6648 - acc: 0.7317 - val_loss: 0.6710 - val_acc: 0.7368
Epoch 3/30
17/17 [==============================] - 0s 3ms/step - loss: 0.6467 - acc: 0.7561 - val_loss: 0.6551 - val_acc: 0.7368
Epoch 4/30
17/17 [==============================] - 0s 2ms/step - loss: 0.6242 - acc: 0.7561 - val_loss: 0.6382 - val_acc: 0.7368
Epoch 5/30
17/17 [==============================] - 0s 3ms/step - loss: 0.5991 - acc: 0.7805 - val_loss: 0.6192 - val_acc: 0.7368
Epoch 6/30
17/17 [==============================] - 0s 5ms/step - loss: 0.5709 - acc: 0.7805 - val_loss: 0.6000 - val_acc: 0.7368
Epoch 7/30
17/17 [==============================] - 0s 3ms/step - loss: 0.5402 - acc: 0.8110 - val_loss: 0.5803 - val_acc: 0.7368
Epoch 8/30
17/17 [==============================] - 0s 3ms/step - loss: 0.5056 - acc: 0.

Epoch 30/30
2/2 [==============================] - 0s 3ms/step
Epoch 1/30
17/17 [==============================] - 1s 8ms/step - loss: 0.6628 - acc: 0.7378 - val_loss: 0.6660 - val_acc: 0.7368
Epoch 2/30
17/17 [==============================] - 0s 2ms/step - loss: 0.6289 - acc: 0.8354 - val_loss: 0.6480 - val_acc: 0.6842
Epoch 3/30
17/17 [==============================] - 0s 3ms/step - loss: 0.5921 - acc: 0.8415 - val_loss: 0.6288 - val_acc: 0.6842
Epoch 4/30
17/17 [==============================] - 0s 3ms/step - loss: 0.5525 - acc: 0.8476 - val_loss: 0.6073 - val_acc: 0.6842
Epoch 5/30
17/17 [==============================] - 0s 3ms/step - loss: 0.5075 - acc: 0.8659 - val_loss: 0.5849 - val_acc: 0.6842
Epoch 6/30
17/17 [==============================] - 0s 2ms/step - loss: 0.4609 - acc: 0.8780 - val_loss: 0.5618 - val_acc: 0.7368
Epoch 7/30
17/17 [==============================] - 0s 3ms/step - loss: 0.4153 - acc: 0.8780 - val_loss: 0.5372 - val_acc: 0.7368
Epoch 8/30
17/17 [=========

## Bag of Words (biagram)

In [5]:
# bag of words + neural nets

# stemming
porter = PorterStemmer() # stemming recovers root words from plurals etc
stemmed_texts = []

X_text = df['annotation_text']

for answer in X_text:
    answer = ' '.join(porter.stem(word) for word in answer.split(' '))
    stemmed_texts.append(answer)

X, y = np.array(stemmed_texts), df['comment_process'] # CHANGE y HERE

#vect = CountVectorizer(ngram_range=(1,1), max_features=1000, stop_words="english") #only unigram 313 features
vect = CountVectorizer(ngram_range=(2,2), max_features=1000, stop_words="english") #only bigram 728 features

X = vect.fit_transform(X).toarray()


# set up storage arrays for each round of validation
roc_auc_scores = np.array([])
accuracy_scores = np.array([])

# split, train, test and store performance metrics
for train_index, test_index in gkf.split(X, y, groups=groups):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    
    model = Sequential()
    model.add(Dense(12, input_shape=(738,), activation='relu')) 
    model.add(Dense(8, activation='relu')) 
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', 
              optimizer='adam',
              metrics = ['acc']
             )
    
    num_epochs = 30
    batch_size = 10
    
    model.fit(
        X_train, 
        y_train, 
        epochs=num_epochs, 
        validation_split=0.1,
        shuffle=True, 
        batch_size=batch_size)
    
    predictions = model.predict(X_test)
    
    # compute some metrics and store them for averaging later on
    roc_auc_scores = np.append(roc_auc_scores, roc_auc_score(y_test, predictions))

# print mean scores for the 5-fold CV
print("average roc_auc score: ", np.round(roc_auc_scores.mean(), 3))
print("stdv roc_auc score: ", np.round(roc_auc_scores.std(), 3))
print("max roc_auc score: ", np.round(roc_auc_scores.max(), 3))

Epoch 1/30
17/17 [==============================] - 1s 8ms/step - loss: 0.6718 - acc: 0.7256 - val_loss: 0.6918 - val_acc: 0.7368
Epoch 2/30
17/17 [==============================] - 0s 3ms/step - loss: 0.6391 - acc: 0.7805 - val_loss: 0.6790 - val_acc: 0.7368
Epoch 3/30
17/17 [==============================] - 0s 3ms/step - loss: 0.6034 - acc: 0.7927 - val_loss: 0.6640 - val_acc: 0.7368
Epoch 4/30
17/17 [==============================] - 0s 3ms/step - loss: 0.5617 - acc: 0.8049 - val_loss: 0.6472 - val_acc: 0.7895
Epoch 5/30
17/17 [==============================] - 0s 3ms/step - loss: 0.5151 - acc: 0.8415 - val_loss: 0.6314 - val_acc: 0.7895
Epoch 6/30
17/17 [==============================] - 0s 3ms/step - loss: 0.4643 - acc: 0.8598 - val_loss: 0.6159 - val_acc: 0.7895
Epoch 7/30
17/17 [==============================] - 0s 3ms/step - loss: 0.4108 - acc: 0.9024 - val_loss: 0.6015 - val_acc: 0.7895
Epoch 8/30
17/17 [==============================] - 0s 3ms/step - loss: 0.3577 - acc: 0.91

Epoch 28/30
17/17 [==============================] - 0s 4ms/step - loss: 0.0239 - acc: 1.0000 - val_loss: 0.6851 - val_acc: 0.7895
Epoch 29/30
17/17 [==============================] - 0s 3ms/step - loss: 0.0212 - acc: 1.0000 - val_loss: 0.6953 - val_acc: 0.7895
Epoch 30/30
2/2 [==============================] - 0s 3ms/step
Epoch 1/30
17/17 [==============================] - 1s 8ms/step - loss: 0.6810 - acc: 0.6707 - val_loss: 0.6894 - val_acc: 0.7368
Epoch 2/30
17/17 [==============================] - 0s 2ms/step - loss: 0.6559 - acc: 0.7744 - val_loss: 0.6826 - val_acc: 0.6842
Epoch 3/30
17/17 [==============================] - 0s 3ms/step - loss: 0.6313 - acc: 0.7805 - val_loss: 0.6757 - val_acc: 0.6316
Epoch 4/30
17/17 [==============================] - 0s 3ms/step - loss: 0.6045 - acc: 0.8232 - val_loss: 0.6694 - val_acc: 0.6316
Epoch 5/30
17/17 [==============================] - 0s 3ms/step - loss: 0.5714 - acc: 0.8293 - val_loss: 0.6639 - val_acc: 0.6316
Epoch 6/30
17/17 [=======

Epoch 30/30
2/2 [==============================] - 0s 3ms/step
Epoch 1/30
17/17 [==============================] - 1s 8ms/step - loss: 0.6839 - acc: 0.7091 - val_loss: 0.6848 - val_acc: 0.6316
Epoch 2/30
17/17 [==============================] - 0s 3ms/step - loss: 0.6643 - acc: 0.7576 - val_loss: 0.6776 - val_acc: 0.6316
Epoch 3/30
17/17 [==============================] - 0s 2ms/step - loss: 0.6425 - acc: 0.7818 - val_loss: 0.6701 - val_acc: 0.6316
Epoch 4/30
17/17 [==============================] - 0s 2ms/step - loss: 0.6157 - acc: 0.7697 - val_loss: 0.6618 - val_acc: 0.6316
Epoch 5/30
17/17 [==============================] - 0s 3ms/step - loss: 0.5821 - acc: 0.7636 - val_loss: 0.6546 - val_acc: 0.6316
Epoch 6/30
17/17 [==============================] - 0s 3ms/step - loss: 0.5407 - acc: 0.7758 - val_loss: 0.6492 - val_acc: 0.6316
Epoch 7/30
17/17 [==============================] - 0s 2ms/step - loss: 0.4916 - acc: 0.8121 - val_loss: 0.6478 - val_acc: 0.6316
Epoch 8/30
17/17 [=========

## Universal Sentece Encoder (USE)

In [6]:
# loading universal sentence encoder
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")

In [7]:
# extract features as X
X = df[['annotation_text','created_by']]

# extract the prediction variable as y
y = df.comment_process # CHANGE y HERE

# set up storage arrays for each round of validation
roc_auc_scores = np.array([])
pred = pd.DataFrame()

# split, train, test and store performance metrics
for train_index, test_index in gkf.split(X, y, groups=groups):

    X_train = X.iloc[train_index].drop(['created_by'], axis=1)
    X_test = X.iloc[test_index].drop(['created_by'], axis=1)
    y_train = y.iloc[train_index]
    y_test = y.iloc[test_index]

    # train classifier on this round of training group
    training_embeddings = embed(X_train.annotation_text.to_list())
    
    model = Sequential()
    model.add(Dense(12, input_shape=(512,), activation='relu')) 
    model.add(Dense(8, activation='relu')) 
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', 
              optimizer='adam',
              metrics = ['acc'])
    
    num_epochs = 30
    batch_size = 10
    
   
    model.fit(
        training_embeddings, 
        y_train, 
        epochs=num_epochs, 
        validation_split=0.1,
        shuffle=True, 
        batch_size=batch_size)
    
    # test classifier on this round of testing group
    testing_embeddings = embed(X_test.annotation_text.to_list())
    predictions = model.predict(testing_embeddings)
    
    # compute some metrics and store them for averaging later on
    roc_auc_scores = np.append(roc_auc_scores, roc_auc_score(y_test, predictions))

    
# print mean scores for the 5-fold CV
print("average roc_auc score: ", np.round(roc_auc_scores.mean(), 3))
print("stdv roc_auc score: ", np.round(roc_auc_scores.std(), 3))
print("max roc_auc score: ", np.round(roc_auc_scores.max(), 3))

Epoch 1/30
17/17 [==============================] - 1s 8ms/step - loss: 0.6808 - acc: 0.7195 - val_loss: 0.6583 - val_acc: 0.7368
Epoch 2/30
17/17 [==============================] - 0s 3ms/step - loss: 0.6369 - acc: 0.7378 - val_loss: 0.6013 - val_acc: 0.7368
Epoch 3/30
17/17 [==============================] - 0s 3ms/step - loss: 0.5790 - acc: 0.7500 - val_loss: 0.5390 - val_acc: 0.7368
Epoch 4/30
17/17 [==============================] - 0s 3ms/step - loss: 0.5212 - acc: 0.8049 - val_loss: 0.4680 - val_acc: 0.7368
Epoch 5/30
17/17 [==============================] - 0s 3ms/step - loss: 0.4588 - acc: 0.8659 - val_loss: 0.4041 - val_acc: 0.7895
Epoch 6/30
17/17 [==============================] - 0s 3ms/step - loss: 0.4025 - acc: 0.8659 - val_loss: 0.3445 - val_acc: 0.8947
Epoch 7/30
17/17 [==============================] - 0s 3ms/step - loss: 0.3533 - acc: 0.9085 - val_loss: 0.2904 - val_acc: 0.9474
Epoch 8/30
17/17 [==============================] - 0s 3ms/step - loss: 0.3119 - acc: 0.91

Epoch 3/30
17/17 [==============================] - 0s 3ms/step - loss: 0.5635 - acc: 0.8354 - val_loss: 0.5485 - val_acc: 0.7368
Epoch 4/30
17/17 [==============================] - 0s 2ms/step - loss: 0.4934 - acc: 0.8659 - val_loss: 0.4868 - val_acc: 0.7895
Epoch 5/30
17/17 [==============================] - 0s 2ms/step - loss: 0.4336 - acc: 0.8659 - val_loss: 0.4292 - val_acc: 0.8421
Epoch 6/30
17/17 [==============================] - 0s 2ms/step - loss: 0.3757 - acc: 0.9024 - val_loss: 0.3718 - val_acc: 0.8947
Epoch 7/30
17/17 [==============================] - 0s 2ms/step - loss: 0.3333 - acc: 0.9207 - val_loss: 0.3175 - val_acc: 0.9474
Epoch 8/30
17/17 [==============================] - 0s 3ms/step - loss: 0.2953 - acc: 0.9268 - val_loss: 0.2897 - val_acc: 0.9474
Epoch 9/30
17/17 [==============================] - 0s 3ms/step - loss: 0.2662 - acc: 0.9390 - val_loss: 0.2457 - val_acc: 0.9474
Epoch 10/30
17/17 [==============================] - 0s 3ms/step - loss: 0.2425 - acc: 0.9

Epoch 5/30
17/17 [==============================] - 0s 2ms/step - loss: 0.5007 - acc: 0.8545 - val_loss: 0.4906 - val_acc: 0.8947
Epoch 6/30
17/17 [==============================] - 0s 3ms/step - loss: 0.4403 - acc: 0.9030 - val_loss: 0.4356 - val_acc: 0.8947
Epoch 7/30
17/17 [==============================] - 0s 2ms/step - loss: 0.3854 - acc: 0.9212 - val_loss: 0.3717 - val_acc: 0.9474
Epoch 8/30
17/17 [==============================] - 0s 3ms/step - loss: 0.3334 - acc: 0.9394 - val_loss: 0.3141 - val_acc: 0.9474
Epoch 9/30
17/17 [==============================] - 0s 2ms/step - loss: 0.2865 - acc: 0.9515 - val_loss: 0.2670 - val_acc: 1.0000
Epoch 10/30
17/17 [==============================] - 0s 3ms/step - loss: 0.2511 - acc: 0.9576 - val_loss: 0.2332 - val_acc: 1.0000
Epoch 11/30
17/17 [==============================] - 0s 2ms/step - loss: 0.2203 - acc: 0.9576 - val_loss: 0.2033 - val_acc: 1.0000
Epoch 12/30
17/17 [==============================] - 0s 2ms/step - loss: 0.1951 - acc: 0

## USE with SMOTE

In [8]:
# extract features as X
X = df[['annotation_text','created_by']]

# extract the prediction variable as y
y = df.comment_process # CHANGE y HERE

# set up storage arrays for each round of validation
roc_auc_scores = np.array([])
pred = pd.DataFrame()

# SMOTE for balancing classes
SEED = 20
smote = SMOTE(random_state=SEED)

# split, train, test and store performance metrics
for train_index, test_index in gkf.split(X, y, groups=groups):

    X_train = X.iloc[train_index].drop(['created_by'], axis=1)
    X_test = X.iloc[test_index].drop(['created_by'], axis=1)
    y_train = y.iloc[train_index]
    y_test = y.iloc[test_index]

    # train classifier on this round of training group
    training_embeddings = embed(X_train.annotation_text.to_list())
    
    # Apply SMOTE
    X_sm, y_sm = smote.fit_resample(training_embeddings.numpy(), y_train)
    
    model = Sequential()
    model.add(Dense(12, input_shape=(512,), activation='relu')) 
    model.add(Dense(8, activation='relu')) 
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', 
              optimizer='adam',
              metrics = ['acc'])
    
    num_epochs = 30
    batch_size = 10
    
   
    model.fit(X_sm, y_sm, epochs=num_epochs, validation_split=0.1, shuffle=True, batch_size=batch_size)
    
    # test classifier on this round of testing group
    testing_embeddings = embed(X_test.annotation_text.to_list())
    predictions = model.predict(testing_embeddings)
    
    # compute some metrics and store them for averaging later on
    roc_auc_scores = np.append(roc_auc_scores, roc_auc_score(y_test, predictions))

    
# print mean scores for the 5-fold CV
print("average roc_auc score: ", np.round(roc_auc_scores.mean(), 3))
print("stdv roc_auc score: ", np.round(roc_auc_scores.std(), 3))
print("max roc_auc score: ", np.round(roc_auc_scores.max(), 3))

Epoch 1/30
25/25 [==============================] - 1s 6ms/step - loss: 0.6750 - acc: 0.7325 - val_loss: 0.6263 - val_acc: 0.9259
Epoch 2/30
25/25 [==============================] - 0s 2ms/step - loss: 0.5979 - acc: 0.9095 - val_loss: 0.5235 - val_acc: 0.9259
Epoch 3/30
25/25 [==============================] - 0s 2ms/step - loss: 0.4973 - acc: 0.9218 - val_loss: 0.4424 - val_acc: 0.8889
Epoch 4/30
25/25 [==============================] - 0s 2ms/step - loss: 0.4036 - acc: 0.9300 - val_loss: 0.3765 - val_acc: 0.8889
Epoch 5/30
25/25 [==============================] - 0s 2ms/step - loss: 0.3262 - acc: 0.9383 - val_loss: 0.2882 - val_acc: 0.9259
Epoch 6/30
25/25 [==============================] - 0s 2ms/step - loss: 0.2708 - acc: 0.9424 - val_loss: 0.2574 - val_acc: 0.9259
Epoch 7/30
25/25 [==============================] - 0s 2ms/step - loss: 0.2261 - acc: 0.9588 - val_loss: 0.2187 - val_acc: 0.9259
Epoch 8/30
25/25 [==============================] - 0s 2ms/step - loss: 0.1950 - acc: 0.96

Epoch 3/30
23/23 [==============================] - 0s 2ms/step - loss: 0.5540 - acc: 0.8761 - val_loss: 0.4612 - val_acc: 0.9615
Epoch 4/30
23/23 [==============================] - 0s 2ms/step - loss: 0.4728 - acc: 0.9071 - val_loss: 0.3669 - val_acc: 0.9615
Epoch 5/30
23/23 [==============================] - 0s 2ms/step - loss: 0.3956 - acc: 0.9027 - val_loss: 0.3053 - val_acc: 0.9615
Epoch 6/30
23/23 [==============================] - 0s 2ms/step - loss: 0.3344 - acc: 0.9248 - val_loss: 0.2473 - val_acc: 0.9615
Epoch 7/30
23/23 [==============================] - 0s 2ms/step - loss: 0.2829 - acc: 0.9336 - val_loss: 0.2139 - val_acc: 0.9615
Epoch 8/30
23/23 [==============================] - 0s 2ms/step - loss: 0.2433 - acc: 0.9425 - val_loss: 0.1800 - val_acc: 0.9615
Epoch 9/30
23/23 [==============================] - 0s 2ms/step - loss: 0.2125 - acc: 0.9513 - val_loss: 0.1703 - val_acc: 0.9615
Epoch 10/30
23/23 [==============================] - 0s 2ms/step - loss: 0.1880 - acc: 0.9

Epoch 5/30
24/24 [==============================] - 0s 2ms/step - loss: 0.5088 - acc: 0.7888 - val_loss: 0.7317 - val_acc: 0.7308
Epoch 6/30
24/24 [==============================] - 0s 2ms/step - loss: 0.4712 - acc: 0.8621 - val_loss: 0.7380 - val_acc: 0.7308
Epoch 7/30
24/24 [==============================] - 0s 2ms/step - loss: 0.4420 - acc: 0.8836 - val_loss: 0.7260 - val_acc: 0.8077
Epoch 8/30
24/24 [==============================] - 0s 2ms/step - loss: 0.4183 - acc: 0.9052 - val_loss: 0.7150 - val_acc: 0.8077
Epoch 9/30
24/24 [==============================] - 0s 2ms/step - loss: 0.4010 - acc: 0.9224 - val_loss: 0.6974 - val_acc: 0.8846
Epoch 10/30
24/24 [==============================] - 0s 2ms/step - loss: 0.3818 - acc: 0.9353 - val_loss: 0.6829 - val_acc: 0.8846
Epoch 11/30
24/24 [==============================] - 0s 2ms/step - loss: 0.3648 - acc: 0.9397 - val_loss: 0.6696 - val_acc: 0.8846
Epoch 12/30
24/24 [==============================] - 0s 2ms/step - loss: 0.3505 - acc: 0

## USE with SMOTE and a Bidirectional LSTM model

In [13]:
# extract features as X
X = df[['annotation_text','created_by']]

# extract the prediction variable as y
y = df.comment_process # CHANGE y HERE

# set up storage arrays for each round of validation
roc_auc_scores = np.array([])
pred = pd.DataFrame()

# SMOTE for balancing classes
SEED = 20
smote = SMOTE(random_state=SEED)

# split, train, test and store performance metrics
for train_index, test_index in gkf.split(X, y, groups=groups):

    X_train = X.iloc[train_index].drop(['created_by'], axis=1)
    X_test = X.iloc[test_index].drop(['created_by'], axis=1)
    y_train = y.iloc[train_index]
    y_test = y.iloc[test_index]

    # train classifier on this round of training group
    training_embeddings = embed(X_train.annotation_text.to_list())
    
    # Apply SMOTE
    X_sm, y_sm = smote.fit_resample(training_embeddings.numpy(), y_train)
    
    model = Sequential()
    model.add(Bidirectional(LSTM(64, return_sequences=True), input_shape=(512, 1)))
    model.add(Bidirectional(LSTM(32))) 
        
    # Dropout for regularization
    model.add(Dropout(0.5))
    
    model.add(Dense(1, activation='sigmoid'))


    model.compile(loss='binary_crossentropy', 
              optimizer='adam',
              metrics = ['acc'])
    
    num_epochs = 30
    batch_size = 10
    
   
    model.fit(X_sm, y_sm, epochs=num_epochs, validation_split=0.1, shuffle=True, batch_size=batch_size)
    
    # test classifier on this round of testing group
    testing_embeddings = embed(X_test.annotation_text.to_list())
    predictions = model.predict(testing_embeddings)
    
    # compute some metrics and store them for averaging later on
    roc_auc_scores = np.append(roc_auc_scores, roc_auc_score(y_test, predictions))

    
# print mean scores for the 5-fold CV
print("average roc_auc score: ", np.round(roc_auc_scores.mean(), 3))
print("stdv roc_auc score: ", np.round(roc_auc_scores.std(), 3))
print("max roc_auc score: ", np.round(roc_auc_scores.max(), 3))

Epoch 1/30
25/25 [==============================] - 9s 201ms/step - loss: 0.6911 - acc: 0.5432 - val_loss: 0.7628 - val_acc: 0.0000e+00
Epoch 2/30
25/25 [==============================] - 4s 167ms/step - loss: 0.6867 - acc: 0.5597 - val_loss: 0.7759 - val_acc: 0.0000e+00
Epoch 3/30
25/25 [==============================] - 4s 165ms/step - loss: 0.7155 - acc: 0.5638 - val_loss: 1.1840 - val_acc: 0.0000e+00
Epoch 4/30
25/25 [==============================] - 4s 166ms/step - loss: 0.7423 - acc: 0.4691 - val_loss: 0.8413 - val_acc: 0.0000e+00
Epoch 5/30
25/25 [==============================] - 4s 163ms/step - loss: 0.6956 - acc: 0.5473 - val_loss: 0.7695 - val_acc: 0.0000e+00
Epoch 6/30
25/25 [==============================] - 4s 170ms/step - loss: 0.7099 - acc: 0.4856 - val_loss: 0.9067 - val_acc: 0.0000e+00
Epoch 7/30
25/25 [==============================] - 4s 166ms/step - loss: 0.7013 - acc: 0.5350 - val_loss: 0.9335 - val_acc: 0.0000e+00
Epoch 8/30
25/25 [==============================

24/24 [==============================] - 9s 200ms/step - loss: 0.6890 - acc: 0.5560 - val_loss: 0.7962 - val_acc: 0.0000e+00
Epoch 2/30
24/24 [==============================] - 4s 161ms/step - loss: 0.6876 - acc: 0.5560 - val_loss: 0.8071 - val_acc: 0.0000e+00
Epoch 3/30
24/24 [==============================] - 4s 162ms/step - loss: 0.6856 - acc: 0.5603 - val_loss: 0.7520 - val_acc: 0.0000e+00
Epoch 4/30
24/24 [==============================] - 4s 163ms/step - loss: 0.6843 - acc: 0.5474 - val_loss: 0.8100 - val_acc: 0.0000e+00
Epoch 5/30
24/24 [==============================] - 4s 163ms/step - loss: 0.7010 - acc: 0.5603 - val_loss: 0.8444 - val_acc: 0.0000e+00
Epoch 6/30
24/24 [==============================] - 4s 163ms/step - loss: 0.6925 - acc: 0.5259 - val_loss: 0.8539 - val_acc: 0.0000e+00
Epoch 7/30
24/24 [==============================] - 4s 167ms/step - loss: 0.6839 - acc: 0.5603 - val_loss: 0.7942 - val_acc: 0.0000e+00
Epoch 8/30
24/24 [==============================] - 4s 160m